# Mapping — Workbook

In this lesson, we're going to learn how to analyze and visualize geographic data.

*Note: You can explore this [workbook](https://mybinder.org/v2/gh/INFO1350/Intro-CA-SP21/master?urlpath=lab/tree/book/07-Mapping/01.5-Mapping-WORKBOOK.ipynb) in the cloud via Binder.*

# Making Interactive Maps

To create interactive maps, we're going to use the Python library [Folium](https://python-visualization.github.io/folium/). Folium is built on top of the popular JavaScript library [Leaflet](https://leafletjs.com/).

### Install Folium

In [ ]:
!pip install folium

### Import Python packages

In [ ]:
import folium
import pandas as pd

### Base Map

First, we need to establish a base map. To do so, we're going to call `folium.Map()`and enter the general latitude/longitude coordinates of the Ithaca area at a particular zoom.

(To find latitude/longitude coordintes for a particular location, you can use Google Maps, [as described here](https://support.google.com/maps/answer/18539?co=GENIE.Platform%3DDesktop&hl=en).)

In [ ]:
base_map = folium.Map(location=[42.44, -76.5], zoom_start=14)
base_map

### Add a Marker

Adding a marker to a map is easy with Folium. We'll simply call `folium.Marker()` at a particular lat/lon, enter some text to display when the marker is clicked on, and then add it to our base map.

In [ ]:
folium.Marker(location=[Latitude?, Longitude?], popup="Intro to Cultural Analytics Class").add_to(base_map)
base_map

What if we wanted to map a bunch of places in the Ithaca area?

In [ ]:
ithaca_df = pd.read_csv("ithaca-places.txt")
ithaca_df

# Geocoding with GeoPy

To mape these places, we need their latitude/longitude coordinates. So we're going to geocode them with the Python package [GeoPy](https://geopy.readthedocs.io/en/stable/#).

GeoPy makes it easier to use a range of third-party [geocoding API services](https://geopy.readthedocs.io/en/stable/#), such as Google, Bing, ArcGIS, and OpenStreetMap.

Though most of these services require an API key, Nominatim, which uses OpenStreetMap data, does not, which is why we're going to use it here.

### Install GeoPy

In [ ]:
!pip install geopy

### Import Nominatim

From GeoPy's list of possible geocoding services, we're going to import Nominatim:

In [ ]:
from geopy.geocoders import Nominatim

### Nominatim & OpenStreetMap

Nominatim (which means "name" in Latin) uses [OpenStreetMap data](https://www.openstreetmap.org/relation/174979) to match addresses with geopgraphic coordinates. Though we don't need an API key to use Nominatim, we do need to create a unique [application name](https://operations.osmfoundation.org/policies/nominatim/). 

Here we're initializing Nominatim as a variable called `geolocator`. Change the application name below to your own application name:

In [ ]:
geolocator = Nominatim(user_agent="YOUR NAME's mapping app", timeout=2)

To geocode an address or location, we simply use the `.geocode()` function:

In [ ]:
location = geolocator.geocode("Cascadilla Street")

In [ ]:
location

### Get Address

In [ ]:
location.address

### Get Latitude and Longitude

In [ ]:
location.latitude, location.longitude

### Get "Importance" Score

In [ ]:
location.raw?

### Get Class and Type

In [ ]:
location.raw?, location.raw?

### Get Multiple Possible Matches

In [ ]:
possible_locations = geolocator.geocode("College Ave", exactly_one=False)

for location in possible_locations:
    print(location.address)
    print(location.latitude, location.longitude)
    print(f"Importance: {location.raw['importance']}\n")

## Your Turn!

## Geocode with Pandas

To geocode every place in a Pandas DataFrame, we can use two options.

### Option 1
First, we can make a list of place dictionaries, and then turn that list into a DataFrame.

Fill in the correct `values` for the `keys` "address," "latitude," "longitude," and "importance" in the dictionary below.

If you need help, look back at how we were accessing this information above. To test your code, see what the `ithaca_df` DataFrame looks like by running the cell below this cell. 

In [ ]:
# Empty list
locations = []

# Loop through every place in the columns "places"
for place in ithaca_df['place']:
    
    #Geolocate the place
    location = geolocator.geocode(place)
    
    # Append a dictionary to the list locations
    locations.append({'place': place,
                      'address': #Your Code Here,
                      'latitude': #Your Code Here,
                      'longitude': #Your Code Here,
                      'importance': #Your Code Here})

In [ ]:
ithaca_df = pd.DataFrame(locations)
ithaca_df

## Option 2

Second, we can create a function, and then `.apply()` that function to the existing DataFrame. This code should run without alteration.

In [ ]:
def find_location(row):
    
    place = row['place']
    location = geolocator.geocode(place)
    
    return location.address, location.latitude, location.longitude, location.raw['importance']

In [ ]:
ithaca_df[['address', 'latitude', 'longitude', 'importance']] = ithaca_df.apply(find_location,
                                                                     axis="columns",
                                                                     result_type="expand")
ithaca_df

### Add Markers From Pandas Data

`folium.Marker(location=[Latitude, Longitude], popup="Place Name").add_to(base_map)`


To add markers for every location in our Pandas dataframe, we can similarly use two options.

### Option 1

We can loop through the DataFrame with `.iterrows()`

In [ ]:
for index, row in ithaca_df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['place']).add_to(base_map)

base_map

### Option 2

Create a function and apply it to the DataFrame

In [ ]:
def create_map_markers(row, map_name):
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['place']).add_to(map_name)

In [ ]:
ithaca_df.apply(create_map_markers, map_name=base_map, axis='columns')

base_map

### Save Map

In [ ]:
base_map.save("Ithaca-map.html")